In [5]:
import pandas as pd
import numpy as np
import os
import re
import matplotlib.pyplot as plt
from xlsxwriter import Workbook

#reading orders and calculating charges and PnL
df = pd.read_csv('orders.csv')


df = df[df['Status']!='REJECTED']

df =df[df['Product']!='CNC']
df['Qty.']=df['Qty.'].apply(lambda x: x.split('/')[0])
df['Qty.']=df['Qty.'].astype(int)
df['Total']=df['Avg. price']*df['Qty.']
df['Gross PnL'] = np.where((df['Type']=='BUY'),-1*df['Total'],df['Total'])
df['Charge']=np.minimum(20,(df['Total']*0.03/100))
df['ETC']=df['Total']*0.003/100
df['GST']=df[['Charge','ETC']].sum(axis='columns')*18/100
df['STT']=np.where(df['Type']=='BUY',0,df['Total']*0.025/100)
df['Stamp Duty']=np.where(df['Type']=='BUY',df['Total']*0.003/100,0)
df['Total Charges']=df[['Charge','ETC','GST','STT','Stamp Duty']].sum(axis='columns')
print(df.info())

#df = df[df['Product']!='CNC']
#df_summary is used as intermediate dataframe
df_summary = pd.DataFrame(df.groupby(['Instrument'])['Gross PnL','Total Charges'].sum()).reset_index()
df_summary['Net PnL']=df_summary['Gross PnL']-df_summary['Total Charges']
df_summary['% Charge']=(df_summary['Total Charges']/df_summary['Gross PnL'])*100
df_summary

#df_total is used as intermediate dataframe
df_total = pd.pivot_table(pd.DataFrame(df_summary.sum(axis=0)).reset_index(),columns=['index'],values=[0],aggfunc=lambda x: ' '.join(str(v) for v in x))
df_total['Instrument']='Total'

#df_final is the final summary dataframe
df_final = pd.concat([df_summary,df_total],axis='rows')
list_columns = ['Gross PnL','Total Charges','Net PnL','% Charge']
for i in list_columns:
    df_final[i]=df_final[i].astype(float)
    df_final[i]=df_final[i].round(2)
print(df_final.info())

writer = pd.ExcelWriter('kite_pnl_calculated.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name='Data', index=False)
df_final.to_excel(writer, sheet_name='Summary', index=False)
writer.save()
df_final

ModuleNotFoundError: No module named 'xlsxwriter'

In [ ]:
import pandas as pd
import numpy as np
import os
import re
import matplotlib.pyplot as plt

In [ ]:
from bs4 import BeautifulSoup as bs

In [ ]:
df = pd.read_csv("orders.csv")

In [ ]:
df.head()

In [ ]:
df['Status'].value_counts()

COMPLETE     8
CANCELLED    3
REJECTED     1
Name: Status, dtype: int64

In [ ]:
df1 = df[df['Status']!='CANCELLED']

In [ ]:
df['Product'].value_counts()

In [ ]:
df2 = df1[df1['Product']!='CNC']

In [ ]:
df2

,Time,Type,Instrument,Product,Qty.,Avg. price,Status
0,2021-12-08 15:15:09,SELL,NTPC,MIS,2000/2000,127.30,COMPLETE
1,2021-12-08 14:40:54,BUY,NTPC,MIS,2000/2000,127.05,COMPLETE
3,2021-12-08 14:15:18,SELL,NTPC,MIS,2300/2300,126.90,COMPLETE
5,2021-12-08 13:29:29,BUY,NTPC,MIS,300/300,126.55,COMPLETE
6,2021-12-08 13:25:12,BUY,NTPC,MIS,2000/2000,126.55,COMPLETE
9,2021-12-08 13:05:13,SELL,NTPC,MIS,2000/2000,126.90,COMPLETE
10,2021-12-08 12:34:44,BUY,NTPC,MIS,1000/1000,126.05,COMPLETE
11,2021-12-08 12:34:15,BUY,NTPC,MIS,1000/1000,126.05,COMPLETE


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Time        12 non-null     object 
 1   Type        12 non-null     object 
 2   Instrument  12 non-null     object 
 3   Product     12 non-null     object 
 4   Qty.        12 non-null     object 
 5   Avg. price  12 non-null     float64
 6   Status      12 non-null     object 
dtypes: float64(1), object(6)
memory usage: 800.0+ bytes


In [ ]:
df.head()

,Time,Type,Instrument,Product,Qty.,Avg. price,Status
0,2021-12-08 15:15:09,SELL,NTPC,MIS,2000/2000,127.30,COMPLETE
1,2021-12-08 14:40:54,BUY,NTPC,MIS,2000/2000,127.05,COMPLETE
2,2021-12-08 14:38:32,BUY,NTPC,MIS,0/2000,126.95,CANCELLED
3,2021-12-08 14:15:18,SELL,NTPC,MIS,2300/2300,126.90,COMPLETE
4,2021-12-08 14:00:41,SELL,NTPC,MIS,0/1300,127.40,CANCELLED


In [ ]:
x = "100/100"
type(x)

str

In [ ]:
x = x.split('/')[0]
int(x)

100

In [ ]:
df2['Qty.']=df2['Qty.'].apply(lambda x: x.split('/')[0])
df2['Qty.']=df2['Qty.'].astype(int)
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8 entries, 0 to 11
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Time        8 non-null      object 
 1   Type        8 non-null      object 
 2   Instrument  8 non-null      object 
 3   Product     8 non-null      object 
 4   Qty.        8 non-null      int32  
 5   Avg. price  8 non-null      float64
 6   Status      8 non-null      object 
dtypes: float64(1), int32(1), object(5)
memory usage: 480.0+ bytes


<ipython-input-20-cb4730453ce5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Qty.']=df2['Qty.'].apply(lambda x: x.split('/')[0])
<ipython-input-20-cb4730453ce5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Qty.']=df2['Qty.'].astype(int)
